In [1]:
import pandas as pd
import numpy as np
import random

from sklearn import model_selection
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
avisos = pd.read_csv('Predecir/avisos_detalles_train.csv')
postulantes = pd.read_csv('Predecir/postulantes_datos_train.csv')
postulaciones = pd.read_csv('Predecir/df_postulaciones.csv')

In [3]:
del postulantes['Unnamed: 0']
del avisos['Unnamed: 0']
del postulaciones['Unnamed: 0']

In [4]:
avisos_si = avisos.copy()
avisos_si['se_postulo'] = 1

In [5]:
postulaciones = postulaciones.sample(frac=0.25)

In [6]:
postulaciones = pd.merge(postulaciones, avisos_si, how='inner', on=['idaviso'])

In [7]:
postulaciones = pd.merge(postulaciones, postulantes, how='inner', on=['idpostulante'])

In [8]:
def random_fill(row):
    return np.random.choice(avisos['idaviso'])

In [9]:
while (len(postulaciones[postulaciones["se_postulo"]==1])*0.5 > len(postulaciones[postulaciones["se_postulo"]==0])):
    relleno = postulantes.sample(frac=0.5).copy()
    relleno['idaviso'] = np.nan
    relleno['idaviso'] = relleno.apply(random_fill, axis=1)
    relleno = pd.merge(relleno, avisos, how='inner', on=['idaviso'])
    postulaciones = pd.merge(postulaciones, relleno, how='outer')
    postulaciones = postulaciones.fillna({'se_postulo' : 0})

In [10]:
postulaciones.shape

(2362836, 46)

In [11]:
postulaciones = postulaciones.dropna()

In [12]:
se_postulo = postulaciones['se_postulo']
del postulaciones['se_postulo']
del postulaciones['idaviso']
del postulaciones['idpostulante']
postulaciones.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,incorp_inmediata,...,total_vistas,vistas_median,cant_dias,nombre,estado,sexo,edad,vistas_postulante,cant_dias_actividad,postu_postulante
0,1,1,41,27,0,0,0,0,0,0,...,3682.0,2.0,34.0,1.0,2.0,1,29.0,91.0,63.0,64.0
1,2,1,41,597,0,0,1,1,0,0,...,6302.0,2.0,21.0,1.0,2.0,1,29.0,91.0,63.0,64.0
2,1,1,5,2257,0,0,0,0,0,0,...,4958.0,2.0,38.0,1.0,2.0,1,29.0,91.0,63.0,64.0
3,2,2,5,1597,0,0,0,0,0,0,...,8308.0,2.0,92.0,1.0,2.0,1,29.0,91.0,63.0,64.0
4,2,2,5,883,0,1,0,0,1,0,...,5276.0,2.0,38.0,1.0,2.0,1,29.0,91.0,63.0,64.0


In [13]:
postulaciones = (postulaciones - postulaciones.mean())/postulaciones.std()

In [14]:
X_train, X_test, y_train, y_test = train_test_split(postulaciones, se_postulo, test_size=0.20, random_state=7)

In [15]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None)

In [16]:
prediction = clf.predict(X_test)

In [17]:
accuracy_score(y_test, prediction, normalize = True)

0.6304818256316236

In [33]:
to_predict  = pd.read_csv('Predecir/test_final_100k.csv')
to_predict  = pd.merge(to_predict, avisos, on = "idaviso",how="left")
to_predict = pd.merge(to_predict, postulantes, on = "idpostulante",how="left")
to_predict = to_predict.drop_duplicates(["idaviso","idpostulante"])

In [34]:
to_predict.shape

(100000, 39)

In [35]:
del to_predict["idaviso"]
del to_predict["idpostulante"]
to_predict = to_predict.dropna()

In [39]:
to_sumit = clf.predict(to_predict.drop("id",axis = "columns"))

In [40]:
to_predict["prediccion"] = to_sumit

In [41]:
res  =  pd.read_csv('Predecir/test_final_100k.csv')

res = pd.merge(res, avisos, on = "idaviso",how="left")
res = pd.merge(res, postulantes, on = "idpostulante",how="left")
res = res.drop_duplicates(["idaviso","idpostulante"])

In [42]:
to_sumit = pd.merge(res,to_predict,on="id",how="left")["prediccion"]
to_sumit = to_sumit.fillna(random.getrandbits(1))
df = pd.DataFrame()
df["sepostulo"] = to_sumit
df["id"] = pd.read_csv('Predecir/test_final_100k.csv')["id"]
final = df.set_index("id")
final.shape

(100000, 1)

In [43]:
final.to_csv("final.csv")